# Internal coordinates and averages

This notebooks focuses on internal coordinates, distances, angle and dihedral angles. The name "internal coordinates" refers to the fact that they are insensitive to global rotations or translations of  the system. 

These internal coordinates will also be used to demonstrate the computation of averages of time series and the error on such averages.

In [ ]:
%matplotlib widget

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import mdtraj
import nglview
import numpy as np
import pandas
from pymbar.timeseries import statisticalInefficiency

The following cell sets up nglview with GUI controls (under development). When clicking on an atom, one can find all its details under the tab `Extra` and then `Picked`. We will often need the atom index in this notebook.

In [ ]:
traj = mdtraj.load("traj.dcd", top="init.pdb")
df = pandas.read_csv("scalars.csv")
view = nglview.show_mdtraj(traj, gui=True)
view.add_representation("ball+stick", selection="protein")
view

## 1. Inter-atomic distances

The following example just computes two bond lengths as function of time, namely for the `N-CA` and `CA-C` bonds in the leucine residue. Note that all indices in Python start counting from zero, so the bond between the first two atoms in the PDB and DCD files is denoted by `[0, 1]`.

The documentation of `compute_distances` can be found here: http://mdtraj.org/1.9.3/api/generated/mdtraj.compute_distances.html#mdtraj.compute_distances

In [ ]:
distances = mdtraj.compute_distances(traj, [[0, 1], [1, 2]])
print(distances.shape)
print(distances[400, 0])
plt.close(1)  # This is needed to rerun the code cell correctly
fig, ax = plt.subplots(num=1)
for counter, col in enumerate(distances.T):
    ax.plot(df["Time (ps)"], col, ".", label=str(counter))
ax.set_xlabel("Time [ps]")
ax.set_ylabel("Distances [nm]")
ax.legend(loc=0)

**<span style="color:#A03;font-size:14pt">
&#x270B; HANDS-ON! &#x1F528;
</span>**

> For the simple bond lengths shown above, the initialization (warming up of the protein during the first 5 picoseconds) seems to have little effect.
> Try to find the atomic indices of strongly electrostatically interacting non-covalent atom pairs, e.g. the nitrogen in the lysine side changes interact with carboxilic groups in a glutamine and the c-terminus. Plot the distances between two pairs of electrostatically interacting atoms. Can you recognize the thermalization in these inter-atomic distances? Add two more distances, one for a hydrogen bond in an alpha helix and one outside an alpha helix.

## 2. Valence and dihedral angles

A similar analysis for the C-S-C angle, the first backbone dihedral angle and a side-chain dihedral angle, is carried out with [compute_angles](http://mdtraj.org/1.9.3/api/generated/mdtraj.compute_angles.html) and [compute_dihedrals](http://mdtraj.org/1.9.3/api/generated/mdtraj.compute_dihedrals.html) in the following code cell.

In [ ]:
angles = mdtraj.compute_angles(traj, [[171, 172, 173]])
dihedrals = mdtraj.compute_dihedrals(traj, [[0, 1, 2, 21], [33, 36, 37, 39]])
print(angles.shape)
plt.close(2)  # This is needed to rerun the code cell correctly
fig, ax = plt.subplots(num=2)
ax.plot(df["Time (ps)"], angles[:, 0] / np.pi * 180, ".", label="C-S-C angle")
ax.plot(
    df["Time (ps)"],
    dihedrals[:, 0] / np.pi * 180,
    ".",
    label="N-CA-C-N angle (psi LEU1)",
)
ax.plot(
    df["Time (ps)"],
    dihedrals[:, 1] / np.pi * 180,
    ".",
    label="CA-CB-CG-OD2 angle (chi2 ASP3)",
)
ax.set_xlabel("Time [ps]")
ax.set_ylabel("Angles [deg]")
ax.legend(loc=0)

Valence angles in proteins are usually rather stiff while the ($\psi$ and $\phi$) dihedrals in the backbone and the $\chi_n$ dihedrals in the side chains explain most of the conformational changes.

## 3. Averages, fluctuations, error on the average

Computing averages and fluctuations (standard deviations) of a time-dependent data series is easily carried out with Numpy. The following code cell contains some examples.

In [ ]:
print("average distance 0 [nm]", distances[:, 0].mean())
print("average distance 0 [nm]", np.mean(distances[:, 0]))
print("all average distances [nm]", distances.mean(axis=0))
print("all average distances [nm]", np.mean(distances, axis=0))
print("st.dev. distance 0 [nm]", distances[:, 0].std())
print("st.dev. distance 0 [nm]", np.std(distances[:, 0]))
print("all st.dev. distances [nm]", distances.std(axis=0))
print("all st.dev. distances [nm]", np.std(distances, axis=0))

The quantities from an MD simulation, of which an average is computed, are stochastic. Hence, any average over a finite number of steps is also a stochastic quantity, subject to an uncertainty. When computing an average over $N$ **uncorrelated** data points, the error on the average is

$$\sigma_{\langle a \rangle} = \sqrt{\frac{1}{N-1}\sum_{i=1}^N (a_i - \langle a \rangle)^2}$$

This formula is rarely useful for computing uncertainties on averages (over time) of quantities from MD simulations, because the values for subsequent time steps are often correlated. (See e.g. the inter-atomic distances for electrostatically interacting pairs of atoms.)

One should correct the factor $1/(N-1)$ and replace $N$ by the number of uncorrelated data points in a time series. The function `num_independent` below estimates this number of independent samples with the block-averaging method. The implementation below is based on the description of the block average method in the book "Computer Simulation of Liquids" by M.P. Allen and D.J. Tildesley, section 6.4.1 on page 192.

In [ ]:
def averror(values, num=None):
    """Compute the error on the average of (un)correlated samples.

    Parameters
    ----------
    values
        samples for which the error on the average is computed.
    num
        The number of independent samples. When not given, the
        default is len(values), which would be correct for
        uncorrelated samples.

    Returns
    -------
    averror
        The error on the averge.

    """
    if num is None:
        num = len(values)
    elif num <= 1:
        return np.nan
    return np.std(values) / np.sqrt(num - 1)


def num_independent(values, fignum=None):
    """Estimate the number of independent samples in a time series.

    This implementation is based on the description in the book
    of Allen and Tildesley. In addition to the book, this code
    attempts to safeguard against balistic motion artifacts when
    performing the extrapolation towards infinite block sizes.

    Parameters
    ----------
    values
        A time-correlated series, must be a numpy array with shape (N,).
    fignum
        A matplotlib figure number for plotting the inefficiency and the
        model used for extrapolation.

    Returns
    -------
    indep
        The number of independent samples. When a constant input is provided,
        or in a few pathological cases, the result will be 1.
    quality
        An empirical quality indicator to judge if the series is sufficiently
        long for a reliable estimate of the number of independent samples.
        This should be at least 5, preferrably more. When less than 5,
        the time series should be made at least 2**(5-quality) times longer.

    """
    if values.ndim != 1:
        raise TypeError("Only one-dimensional arrays are supported.")

    # At least 16 blocks, then halve size at each iteration.
    bss = []
    ineffs = []
    bs = len(values) // 16
    while bs >= 1:
        nb = len(values) // bs
        blocks = values[: nb * bs].reshape(nb, bs)
        avvar = blocks.mean(axis=1).var(ddof=1) / (nb - 1)
        bss.insert(0, bs)
        ineffs.insert(0, avvar)
        bs //= 2

    # Return in case of no blocks
    if len(bss) < 1:
        return 1, 0

    # Convert to inefficiencies.
    bss = np.array(bss)
    ineffs = np.array(ineffs)
    ineffs /= ineffs[0]

    # Fit a simple model:  y = a * x / (a + x - 1)
    # Discard decreasing part of the inefficiency, e.g. due to balistic motion.
    i0 = ineffs.argmin()
    y = ineffs[i0:] / ineffs[i0]
    x = bss[i0:] / bss[i0]
    # At least two data points are needed for the fit.
    if len(y) >= 2:
        dm = (y - x) / x
        ev = (y * (1 - x)) / x
        a = np.dot(ev, dm) / np.dot(dm, dm)
        ineff_limit = a * ineffs[i0]
        optbs = a * bss[i0]
        ineffs_model = ineff_limit * x / (a + x - 1)
    else:
        ineff_limit = 0
        optbs = len(values)
        ineffs_model = None

    if ineff_limit <= 0 or ineff_limit > len(values):
        num_indep = 1
    else:
        num_indep = len(values) / ineff_limit

    # The quality measure is the minimum of the number of points
    # used in the fit and the number of points after the optimal
    # block size.
    quality = min(len(y), (bss > optbs).sum())

    if fignum is not None:
        plt.close(fignum)
        fig, ax = plt.subplots(num=fignum)
        ax.set_title(
            f"Statistical inefficiency: {ineff_limit:.1f} ||"
            f"Independent samples: {num_indep:.1f} ||"
            f"Quality: {quality:d}"
        )
        ax.plot(bss, ineffs, "o", color="#aaaaaa")
        ax.plot(bss[i0:], ineffs[i0:], "ko")
        if ineffs_model is not None:
            ax.plot(bss[i0:], ineffs_model, "-", color="C0")
            ax.axhline(ineff_limit, color="C0", ls=":")
            ax.axvline(optbs, color="C0", ls=":")
        ax.set_xscale("log")
        ax.set_xlabel("Block size")
        ax.set_ylabel("Statistical inefficiency")

    return num_indep, quality


values = distances[:, 0]
# values = angles[:, 0]
# values = dihedrals[:, 0]
num_indep, quality = num_independent(values, fignum=3)
print("Quality:", quality)
print("Number of values:", len(values))
print("Number of independent values:", num_indep)
print("Error on the average:", averror(values, num_indep))

**<span style="color:#A03;font-size:14pt">
&#x270B; HANDS-ON! &#x1F528;
</span>**

> Try estimating the error on the average of all internal coordinates computed so far. Does the number of independent samples correlate with the time dependence plotted previously?

## 4. Estimating the initialization phase (or burn-in)

The error estimation in the previous section only works well when the provided time series is taken from an simulation in equilibrium, i.e. without initialization phase. One could try to estimate the length of the initialization or burn-in phase by visually inspecting the time-dependency in a plot. However, this is tedious, subjective and error-prone practice. In this section, we will employ the scheme proposed in [10.1021/acs.jctc.5b00784](https://doi.org/10.1021/acs.jctc.5b00784) to determine the initialization phase automatically. This scheme prescribes that the optimal initialization phase maximizes the number of independent samples in the remainder.

In [ ]:
def scan_initialization(values):
    """Determine initialization phase.

    This is a very approximate maximization of the number of independent
    samples. A more fine-grained maximization is probably not that useful.
    """
    ninit = 0
    num_indep = num_independent(values)[0]
    ninit_new = 1
    while ninit_new < len(values):
        num_indep_new = num_independent(values[ninit_new:])[0]
        if num_indep_new > num_indep:
            ninit = ninit_new
            num_indep = num_indep_new
        ninit_new = max(ninit_new + 1, int(ninit_new * 1.1))
    return ninit


values = distances[:, 0]
# values = angles[:, 0]
# values = dihedrals[:, 0]
# values = df["Potential Energy (kJ/mole)"].to_numpy()
ninit = scan_initialization(values)

# Plot the time dependence and the initialization part
plt.close(4)  # This is needed to rerun the code cell correctly
fig, ax = plt.subplots(num=4)
ax.plot(df["Time (ps)"][:ninit], values[:ninit], ".")
ax.plot(df["Time (ps)"][ninit:], values[ninit:], ".")
ax.axvline(df["Time (ps)"][ninit], color="k")
ax.set_xlabel("Time [ps]")

# Exhaustive plot of the number of independent samples as function
# of the initialization time.
tmp = np.array([num_independent(values[ninit:]) for ninit in range(len(values))])
num_indeps, qualities = tmp.T
plt.close(5)  # This is needed to rerun the code cell correctly
fig, ax1 = plt.subplots(num=5)
ax1 = plt.gca()
ax1.set_xlabel("Initialization phase [1]")
ax1.set_ylabel("Quality of the estimate", color="C0")
ax1.plot(qualities, color="C0")
ax1.axvline(ninit, color="k")
ax1.tick_params(axis="y", labelcolor="C0")
ax2 = ax1.twinx()
ax2.set_ylabel("Number of independent samples", color="C2")
ax2.plot(num_indeps, color="C2", lw=1)
ax2.tick_params(axis="y", labelcolor="C2")

**<span style="color:#A03;font-size:14pt">
&#x270B; HANDS-ON! &#x1F528;
</span>**

> Repeat the above code a few different time series and answer the following questions:
>
> - Visual inspection of the initialization phase can be too optimistic or pessimistic?
> - What is the accuracy of an error estimate?
>
> An important consideration with the last question is that one has no information of unseen states in an MD simulation. For example, a finite MD simulation may be sampling a metastable conformation without relaxing to a more stable one. Any averages and error estimates derived from such a run are biased because they are not considering the thermodynamically dominant conformation.

Note that in the reference [10.1021/acs.jctc.5b00784](https://doi.org/10.1021/acs.jctc.5b00784), an alternative (more robust but not as easy to understand) function is recommended to compute the number of independent samples in a time series. The original implementation of the authors can be found in [pymbar](https://pymbar.readthedocs.io/en/master/).

In [ ]:
print(statisticalInefficiency(distances[:, 0]))
print(statisticalInefficiency(angles[:, 0]))
print(statisticalInefficiency(dihedrals[:, 0]))

## 5. Average of a dihedral angle

The average of a dihedral angle can become meaningless because these angles are subject to a periodic boundary, i.e. $\phi=35^\circ$ and $\phi=395^\circ$ represent the same dihedral angles. As a consequence, dihedral angles exceeding $180^\circ$ will continue at $-180^\circ$. Such discontinuous jumps result in meaningless averages.

This problem can be surmounted by computing a slightly different type of average, i.e. the average cosine and sine of the angle, followed by a conversion back into an angle. For example:

In [ ]:
x = np.cos(dihedrals[:, 1]).mean()
y = np.sin(dihedrals[:, 1]).mean()
avdihed1 = np.arctan2(y, x)
avdihed1_wrong = dihedrals[:, 1].mean()
print(avdihed1 / np.pi * 180)
print(avdihed1_wrong / np.pi * 180)
plt.close(6)  # This is needed to rerun the code cell correctly
fig, ax = plt.subplots(num=6)
ax.plot(
    df["Time (ps)"],
    dihedrals[:, 1] / np.pi * 180,
    ".",
    label="CA-CB-CG-OD2 angle (chi2 ASP3)",
)
ax.axhline(avdihed1 / np.pi * 180, color="k")
ax.axhline(avdihed1_wrong / np.pi * 180, color="r")
ax.set_xlabel("Time [ps]")
ax.set_ylabel("Angles [deg]")
ax.legend(loc=0)

In this case, the difference may be small but is certainly noticeable. The naive calculation of the average includes some samples close to $-180^\circ$, whereas the alternative approach does not suffer from this issue. To understand why this works, consider a plot of the sine versus the cosine:

In [ ]:
plt.close(7)  # This is needed to rerun the code cell correctly
fig, ax = plt.subplots(num=7)
ax.plot(np.cos(dihedrals[:, 1]), np.sin(dihedrals[:, 1]), "k+", alpha=0.1)
ax.plot([x], [y], "ro")
ax.plot([0, x], [0, y], "r-")
ax.add_patch(
    mpatches.Arc([0, 0], 1, 1, angle=0, theta1=0, theta2=avdihed1 / np.pi * 180, color="r")
)
ax.axhline(0, color="k")
ax.axvline(0, color="k")
ax.set_xlim(-1.1, 1.1)
ax.set_ylim(-1.1, 1.1)
ax.set_aspect("equal")

When plotting the dihedral angles as points on a circle, i.e. $x=\cos\phi$ and $y=\sin\phi$, there is no longer a discontinuity and the average is well-behaved. The angle is then derived from the average (red dot) by computing the angle with the X-axis, using the `arctan2` function. A few remarks:

- The function `arctan2` takes the Y-coordinate as first argument.

- This approach will not work when the dihedral angles uniformly distributed over the interval $[-180^\circ,180^\circ]$. In this case, the average cosine and sine are (nearly) zero and the angle with the X-axis is not (or ill) defined.

## 6. Histogram and free energy

Histograms may also be convenient to characterize an internal coordinate. We will apply it here to the C-S-C angle, which is a relatively stiff mode. The angles are also [normally distributed](https://en.wikipedia.org/wiki/Normal_distribution), which can be shown by overlaying the histogram with the normal probability density.

For reference, the force-field parameters for this angle can be found here:

https://github.com/openmm/openmm/blob/master/wrappers/python/openmm/app/data/amber14/protein.ff14SB.xml#L3077

and reads

```
<Angle angle="1.726130630222392" k="518.816" type1="protein-CT" type2="protein-S" type3="protein-CT"/>
```

The equilibrium angle in degrees is $1.726130630222392 \text{ rad}$, which corresponds to $98.9^\circ$.

In [ ]:
plt.close(8)  # This is needed to rerun the code cell correctly
fig, ax = plt.subplots(num=8)
a = angles / np.pi * 180  # for convenience
bins = np.arange(85, 115, 1)
ax.hist(a, bins, density=True)
ava = a.mean()
avs = a.std()
x = np.linspace(80, 120, 79)
# The following equation is the probability density of a
# univariate normal distribution.
y = np.exp(-((x - ava) ** 2) / (2 * avs**2)) / np.sqrt(2 * avs**2 * np.pi)
ax.plot(x, y)
ax.axvline(ava, color="k")
ax.set_title(f"Average = {ava:.1f}, Std.Dev. = {avs:.1f}")
ax.set_xlabel("C-S-C angle [deg]")
ax.set_ylabel("Probability density [1/deg]")

The average is slightly different from the equilibrium value, which may be due to other force-field terms influencing the local geometry.

**<span style="color:#A03;font-size:14pt">
&#x270B; HANDS-ON! &#x1F528;
</span>**

> How can you judge if the average and the equilibrium are significantly different?

The empirical probabilities from the histogram can be translated into free energies (up to a constant) by employing the following relationship:

$$F(\theta) = -k_B T \log(p(\theta))$$

This relation can also be applied to the probability density to obtain a quadratic approximation of the free energy.

In [ ]:
boltzmann = 1e-3 * 1.380649e-23 * 6.02214076e23  # in kJ/mol
temperature = 300
# Force field parameters
theta0_par = 1.726130630222392 / np.pi * 180  # rest angle converted to degrees
# force constant converted to kJ/mol/degree^2
fc_par = 518.816 / (180 / np.pi) ** 2
f0 = 0.5 * fc_par * (x - theta0_par) ** 2
# Quadratic approximation
fc = boltzmann * temperature / avs**2  # force constant
f = 0.5 * fc * (x - ava) ** 2
# Empiricial model, using histogram
pe = np.histogram(a, bins, density=True)[0]
with np.errstate(divide="ignore"):
    fe = -boltzmann * temperature * np.log(pe)
fe -= fe.min()

plt.close(9)  # This is needed to rerun the code cell correctly
fig, ax = plt.subplots(num=9)
ax.plot(x, f0, label="Force field valence angle term")
ax.plot(x, f, label="Quadratic free energy model")
ax.plot((bins[1:] + bins[:-1]) / 2, fe, label="Empirical free energy")
ax.set_xlabel("C-S-C angle [deg]")
ax.set_ylabel("Free energy [kJ/mol]")
ax.legend(loc=0)

# Finally, the force constant is printed in kJ/mol/rad^2, for comparison with the force-field parameter.
print(fc * (180 / np.pi) ** 2)

Also here, the free-energy force constant deviates slightly from the AMBER force-field parameter. Again, this could be due to an influence from the environment of the C-S-C angle.

**<span style="color:#A03;font-size:14pt">
&#x270B; HANDS-ON! &#x1F528;
</span>**

> Also estimate the error on the force constant.